In [1]:
import pandas as pd
import numpy as np
from mortgage import Loan
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from pickle import load

# For reproducibility
np.random.seed(42)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils

Using TensorFlow backend.


In [2]:
# load in HUD 2020 FMR 50 data (median rent by county)
rent_data = pd.read_excel('../data/data.gov/FY2020_50_County.xlsx')

In [3]:
rent_data.head()

,fips2010,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,state,cbsasub20,areaname20,county,cousub,cntyname,name,pop2017,hu2017,state_alpha
0,100199999,615,741,876,1105,1504,1,METRO33860M33860,"Montgomery, AL MSA",1,99999,Autauga County,Autauga County,55035,23152,AL
1,100399999,806,811,992,1355,1696,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3,99999,Baldwin County,Baldwin County,203360,110023,AL
2,100599999,508,512,674,840,985,1,NCNTY01005N01005,"Barbour County, AL",5,99999,Barbour County,Barbour County,26200,11918,AL
3,100799999,875,937,1073,1404,1551,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7,99999,Bibb County,Bibb County,22580,9112,AL
4,100999999,875,937,1073,1404,1551,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9,99999,Blount County,Blount County,57665,24161,AL


In [10]:
rent_data.iloc[0]['cntyname']

'Autauga County'

In [11]:
# load fips data
fips = pd.read_csv('../data/fips.csv',dtype={'county_code':str})

In [73]:
#data = pd.read_csv('../data/mortgage_model.csv')

In [12]:
# load model
model = load_model('../models/binary_nn.h5')
# load scaler
ss = load(open('../models/scaler.pkl', 'rb'))

/Users/scottrosengrants/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/scottrosengrants/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [95]:
# create a placeholder user df

user = {
 'county_code': [0],
 'census_tract': [0],
 'preapproval': [0],
 'loan_amount': [0],
 'combined_loan_to_value_ratio': [0],
 'interest_rate': [0],
 'loan_term': [0],
 'interest_only_payment': [0],
 'balloon_payment': [0],
 'property_value': [0],
 'occupancy_type': [0],
 'total_units': [0],
 'income': [0],
 'submission_of_application': [0],
 'tract_population': [0],
 'ffiec_msa_md_median_family_income': [0],
 'tract_to_msa_income_percentage': [0],
 'tract_owner_occupied_units': [0],
 'tract_one_to_four_family_homes': [0],
 'tract_median_age_of_housing_units': [0],
 'state_code_AL': [0],
 'state_code_AR': [0],
 'state_code_AZ': [0],
 'state_code_CA': [0],
 'state_code_CO': [0],
 'state_code_CT': [0],
 'state_code_DC': [0],
 'state_code_DE': [0],
 'state_code_FL': [0],
 'state_code_GA': [0],
 'state_code_GU': [0],
 'state_code_HI': [0],
 'state_code_IA': [0],
 'state_code_ID': [0],
 'state_code_IL': [0],
 'state_code_IN': [0],
 'state_code_KS': [0],
 'state_code_KY': [0],
 'state_code_LA': [0],
 'state_code_MA': [0],
 'state_code_MD': [0],
 'state_code_ME': [0],
 'state_code_MI': [0],
 'state_code_MN': [0],
 'state_code_MO': [0],
 'state_code_MS': [0],
 'state_code_MT': [0],
 'state_code_NC': [0],
 'state_code_ND': [0],
 'state_code_NE': [0],
 'state_code_NH': [0],
 'state_code_NJ': [0],
 'state_code_NM': [0],
 'state_code_NV': [0],
 'state_code_NY': [0],
 'state_code_OH': [0],
 'state_code_OK': [0],
 'state_code_OR': [0],
 'state_code_PA': [0],
 'state_code_PR': [0],
 'state_code_RI': [0],
 'state_code_SC': [0],
 'state_code_SD': [0],
 'state_code_TN': [0],
 'state_code_TX': [0],
 'state_code_UT': [0],
 'state_code_VA': [0],
 'state_code_VT': [0],
 'state_code_WA': [0],
 'state_code_WI': [0],
 'state_code_WV': [0],
 'state_code_WY': [0],
 'loan_type_2': [0],
 'loan_type_3': [0],
 'loan_type_4': [0]
}

user_df = pd.DataFrame.from_dict(user)

In [96]:
user_df

,county_code,census_tract,preapproval,loan_amount,combined_loan_to_value_ratio,interest_rate,loan_term,interest_only_payment,balloon_payment,property_value,...,state_code_UT,state_code_VA,state_code_VT,state_code_WA,state_code_WI,state_code_WV,state_code_WY,loan_type_2,loan_type_3,loan_type_4
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Cash on Cash Return

Monthly Cashflow = Monthly Total Income - Vacancy Rate - Expense Reserves - Monthly Expenses(water & trash) - Monthly Financing Cost - Monthly Taxes - Monthly Insurance

Annual Cashflow = Monthly Cashflow * 12
Cash on Cash = Annual Cashflow / Total Cash Invested

### Cap Rate

(mti - vr - er - me - mt - mi) * 12 / price

### Monthly
mti = Monthly Total Income (total income of property including all units and aux services)

vr = Vacancy Rate (percentage of income held out to cover costs of vacant unit, typically 5% of mti) 

er = Expense Reserves (Maintenance and Capital Improvements, usually 10-25% of mti)

me = Monthly Expenses (landlord paid expenses, typically water and trash)

mfc = Monthly Financing Cost (principle and interest payments)

mt = Monthly Taxes (annual tax / 12)

mi = Monthly Insurance (annual insurance cost / 12) 

mcf = Monthly Cashflow (mti - vr - er - me - mfc - mt - mi)

### Annual
acf = Annual Cashflow (mcf * 12)

### Financial
tci = Total Cash Invested (amount of money initially put into deal

coc = Cash On Cash (acf / tci) * 100 


## House API
from atom_api_final notebook

In [15]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize

def call_api_all(bld_num,street,str_type,unit=None,unit_num=None,city=None,state=None):
    import http.client
    
    conn = http.client.HTTPSConnection("api.gateway.attomdata.com") 

    headers = { 
        'accept': "application/json", 
        'apikey': "5b3709e8f052f5a495b91df054a47709", 
    } 
    address_1 = f"address1={bld_num}%20{street}%20{str_type}%20{unit}%20{unit_num}"
    address_2 = f"&address2={city}%2C%20{state}"
    conn.request("GET", f"/propertyapi/v1.0.0/allevents/detail?{address_1}{address_2}", headers=headers) 

    res = conn.getresponse() 
    data = res.read() 
    
#    print(data.decode("utf-8"))
    return data
#    print("GET", f"/propertyapi/v1.0.0/property/detail?{address}", "headers=headers")

def extract_details(api_data):
    house = json.loads(api_data)
    
    # create new dfs based on dictionary keys

    # df_1 ['obPropId', 'fips', 'apn', 'apnOrig', 'attomId']
    df_1 = pd.DataFrame(house['property'][0]['identifier'],index=[0])

    # df_2 ['depth', 'frontage', 'lotnum', 'lotsize1', 'lotsize2', 'pooltype']
    df_2 = pd.DataFrame(house['property'][0]['lot'],index=[0])

    # df_3 ['blockNum', 'countrysecsubd', 'countyuse1', 'muncode', 'munname',
    #       'srvyRange', 'srvySection', 'srvyTownship', 'subdname', 'subdtractnum','taxcodearea']
    df_3 = pd.DataFrame(house['property'][0]['area'],index=[0])

    # df_4 ['country', 'countrySubd', 'line1', 'line2', 'locality', 'matchCode',
    #       'oneLine', 'postal1', 'postal2', 'postal3']
    df_4 = pd.DataFrame(house['property'][0]['address'],index=[0])

    # df_5 = ['accuracy', 'elevation', 'latitude', 'longitude', 'distance', 'geoid']
    df_5 = pd.DataFrame(house['property'][0]['location'],index=[0])

    # df_6 ['absenteeInd', 'propclass', 'propsubtype', 'proptype', 'yearbuilt',
    #       'propLandUse', 'propIndicator', 'legal1']
    df_6 = pd.DataFrame(house['property'][0]['summary'],index=[0])

    #df_7 ['heatingtype']
    df_7 = pd.DataFrame(house['property'][0]['utilities'],index=[0])

    #df_8 **['size', 'rooms', 'interior', 'construction', 'parking', 'summary']** dict keys
    df_8 = pd.DataFrame(house['property'][0]['building'],index=[0])

    #df_9 ['lastModified', 'pubDate']
    df_9 = pd.DataFrame(house['property'][0]['vintage'],index=[0])
    
    # create new dfs based on df_8 dictionary keys

    # df_b1 ['bldgsize', 'grosssize', 'grosssizeadjusted', 'groundfloorsize',
    #       'livingsize', 'sizeInd', 'universalsize']
    df_b1 = pd.DataFrame(house['property'][0]['building']['size'],index=[0])

    # df_b2 ['bathfixtures', 'baths1qtr', 'baths3qtr', 'bathscalc', 'bathsfull',
    #       'bathshalf', 'bathstotal', 'beds', 'roomsTotal']
    df_b2 = pd.DataFrame(house['property'][0]['building']['rooms'],index=[0])

    #df_b3 ['bsmtsize', 'bsmttype', 'fplccount', 'fplcind', 'fplctype']
    df_b3 = pd.DataFrame(house['property'][0]['building']['interior'],index=[0])

    #df_b4 ['condition', 'wallType']
    df_b4 = pd.DataFrame(house['property'][0]['building']['construction'],index=[0])

    #df_b5 ['garagetype', 'prkgSize', 'prkgSpaces', 'prkgType']
    df_b5 = pd.DataFrame(house['property'][0]['building']['parking'],index=[0])

    #df_b6 ['bldgsNum', 'bldgType', 'imprType', 'levels', 'mobileHomeInd',
    #       'quality', 'storyDesc', 'unitsCount', 'yearbuilteffective']
    df_b6 = pd.DataFrame(house['property'][0]['building']['summary'],index=[0])
    
    ### AVM
    #df_v1 ['scr', 'value', 'high', 'low', 'valueRange']
    df_v1 = pd.DataFrame(house['property'][0]['avm']['amount'],index=[0])
    
    #df_v2 ['avmpoorlow', 'avmpoorhigh', 'avmpoorscore', 'avmgoodlow', 'avmgoodhigh', 
    #      'avmgoodscore', 'avmexcellentlow', 'avmexcellenthigh', 'avmexcellentscore']
    df_v2 = pd.DataFrame(house['property'][0]['avm']['condition'],index=[0])
    
    #df_v3 ['taxamt', 'taxpersizeunit', 'taxyear']
    df_v3 = pd.DataFrame(house['property'][0]['assessment']['tax'],index=[0])
    
    # Build df for property 1
    prop_1 = pd.concat([df_1['fips'],df_2,df_3,df_4,df_5,df_6,df_7,df_9,df_b1,
                        df_b2,df_b3,df_b4,df_b5,df_b6,df_v1,df_v2,df_v3],axis=1)
    
    return prop_1

In [43]:
def get_rent(bed_count,state, county):
    row = rent_data[(rent_data['state_alpha'] == str(state)) & (rent_data['cntyname'] == county)]
    if bed_count < 0:
        return 'Must enter bedroom count greater than zero'
        
    elif bed_count <= 4:
        return int(row['rent50_'+str(bed_count)])
    
    else: 
        extra_beds = bed_count - 4
        rent = row['rent50_4']
        for i in range(extra_beds):
            rent = rent * 1.15
        return int(rent)
    

In [47]:
get_rent(5,'CO','Denver County')

3106

In [23]:
def coc(df,principal,cash):
    
    #calculate loan
    loan = Loan(principal, interest=.04, term=30)
    pymt = loan.monthly_payment
    
    #calculate re figures
    mti = float(df['rent'].loc[0])
    vr = float(mti * .05)
    er = float(mti * (.05 + .05))
    me = float((100 + 50)) #water & trash
    mfc = float(pymt)
    mt = float(df['taxamt'].loc[0]/12)
    mi = float(mt)  #(3000/12)
    # Composite
    mcf = mti - vr - er - me - mfc - mt - mi

    # Annual
    acf = np.round((mcf * 12),2)

    # Financial 
    tci = cash
    coc = np.round(((acf / tci) * 100),2)
    
    print(f'Your Annual Cashflow is ${acf} giving you a Cash-On-Cash return of {coc}%!')
    print(f'{mcf} = {mti} - {vr} - {er} - {me} - {mfc} - {mt} - {mi}')
    print(f'Cash Flow = Rent Income - Vacancy - Maint/Cap Exp - Water&Trash - Finance Exp - Tax - Insurance')
    
    coc_dict = {'mti':mti,
               'vr':vr,
               'er':er,
               'me':mfc,
               'mfc':mfc,
               'mt':mt,
               'mi':mi,
               'mcf':mcf,
               'acf':acf,
               'tci':tci,
               'coc':coc}
    return coc_dict

In [32]:
def results(principal, cash, bld_num,street,str_type,unit=None,unit_num=None,city=None,state=None):
    prop = extract_details(call_api_all(bld_num,street,str_type,unit,unit_num,city,state))
    prop_sm = prop[['beds','taxamt','countrySubd','countrysecsubd']].copy() 
    prop_sm['rent'] = get_rent(prop_sm['beds'].loc[0],prop_sm['countrySubd'].loc[0],prop_sm['countrysecsubd'].loc[0])
    prop['rent'] = prop_sm['rent']
    coc_dict = coc(prop_sm,principal,cash)
    return prop, coc_dict

In [33]:
#10226 Noland Rd, Lenexa, KS 66215
#$265,000
#https://www.zillow.com/homedetails/10226-Noland-Rd-Lenexa-KS-66215/75676324_zpid/
prop_1, prop_1_coc = results(200_000,65_000,'10226','Noland','Rd','Lenexa','KS')

Your Annual Cashflow is $-855.54 giving you a Cash-On-Cash return of -1.32%!
-71.29499999999996 = 1837.0 - 91.85000000000001 - 183.70000000000002 - 150.0 - 954.83 - 263.9575 - 263.9575
Cash Flow = Rent Income - Vacancy - Maint/Cap Exp - Water&Trash - Finance Exp - Tax - Insurance


In [50]:
prop_1['avmpoorlow'],prop_1['avmpoorhigh']

(0    228000.0
 Name: avmpoorlow, dtype: float64, 0    282150.0
 Name: avmpoorhigh, dtype: float64)

In [30]:
prop_1.columns

Index(['fips', 'depth', 'frontage', 'lotnum', 'lotsize1', 'lotsize2',
       'countrysecsubd', 'countyuse1', 'muncode', 'munname', 'subdname',
       'country', 'countrySubd', 'line1', 'line2', 'locality', 'matchCode',
       'oneLine', 'postal1', 'postal2', 'postal3', 'accuracy', 'elevation',
       'latitude', 'longitude', 'distance', 'geoid', 'absenteeInd',
       'propclass', 'proptype', 'yearbuilt', 'propLandUse', 'propIndicator',
       'legal1', 'coolingtype', 'heatingtype', 'lastModified', 'pubDate',
       'bldgsize', 'grosssize', 'grosssizeadjusted', 'groundfloorsize',
       'livingsize', 'sizeInd', 'universalsize', 'bathfixtures', 'baths1qtr',
       'baths3qtr', 'bathscalc', 'bathsfull', 'bathshalf', 'bathstotal',
       'beds', 'roomsTotal', 'bsmtsize', 'fplccount', 'fplcind', 'fplctype',
       'condition', 'constructiontype', 'foundationtype', 'frameType',
       'roofcover', 'wallType', 'prkgSize', 'archStyle', 'bldgsNum', 'levels',
       'mobileHomeInd', 'unitsCount'

In [29]:
prop_1_coc

{'mti': 1837.0,
 'vr': 91.85000000000001,
 'er': 183.70000000000002,
 'me': 954.83,
 'mfc': 954.83,
 'mt': 263.9575,
 'mi': 263.9575,
 'mcf': -71.29499999999996,
 'acf': -855.54,
 'tci': 65000,
 'coc': -1.32}

In [82]:
# check a property with a floating zero in the county code
prop_2, prop_2_coc = results(400_000,150_000,'1232','Quaker','Street','Golden','CO')

Your Annual Cashflow is $-1339.72 giving you a Cash-On-Cash return of -0.89%!
-111.64333333333349 = 2701.0 - 135.05 - 270.1 - 150.0 - 1909.66 - 173.91666666666666 - 173.91666666666666
Cash Flow = Rent Income - Vacancy - Maint/Cap Exp - Water&Trash - Finance Exp - Tax - Insurance


In [83]:
prop_2.columns

Index(['fips', 'depth', 'frontage', 'lotnum', 'lotsize1', 'lotsize2',
       'pooltype', 'countrysecsubd', 'countyuse1', 'muncode', 'munname',
       'srvyRange', 'srvySection', 'srvyTownship', 'subdname', 'subdtractnum',
       'taxcodearea', 'country', 'countrySubd', 'line1', 'line2', 'locality',
       'matchCode', 'oneLine', 'postal1', 'postal2', 'postal3', 'accuracy',
       'elevation', 'latitude', 'longitude', 'distance', 'geoid',
       'absenteeInd', 'propclass', 'propsubtype', 'proptype', 'yearbuilt',
       'propLandUse', 'propIndicator', 'legal1', 'heatingtype', 'lastModified',
       'pubDate', 'bldgsize', 'grosssize', 'grosssizeadjusted',
       'groundfloorsize', 'livingsize', 'sizeInd', 'universalsize',
       'bathfixtures', 'baths1qtr', 'baths3qtr', 'bathscalc', 'bathsfull',
       'bathshalf', 'bathstotal', 'beds', 'roomsTotal', 'bsmtsize', 'bsmttype',
       'fplccount', 'fplcind', 'fplctype', 'constructiontype', 'frameType',
       'garagetype', 'prkgSize', 'prkgSp

In [84]:
fips['census_tract'].dtypes

dtype('float64')

In [97]:
def apply_census(fips_num, user_df):
    mask = fips['county_code'] == fips_num
    df = fips[mask]
    user_df['county_code'] = fips_num
    user_df['census_tract'] = int(df['census_tract'])
    user_df['tract_population'] = int(df['tract_population'])
    user_df['ffiec_msa_md_median_family_income'] = float(df['ffiec_msa_md_median_family_income'])
    user_df['tract_to_msa_income_percentage'] = float(df['tract_to_msa_income_percentage'])
    user_df['tract_owner_occupied_units'] = float(df['tract_owner_occupied_units'])
    user_df['tract_one_to_four_family_homes'] = float(df['tract_one_to_four_family_homes'])
    user_df['tract_median_age_of_housing_units'] = float(df['tract_median_age_of_housing_units'])
    return user_df

In [98]:
user_df = apply_census(prop_2['fips'].loc[0], user_df)

In [99]:
def add_units(property_df, user_df):
    user_df['total_units'] = int(property_df['unitsCount'])
    return user_df

In [100]:
user_df = add_units(prop_2, user_df)

In [101]:
def add_state(property_df, user_df):
    column = 'state_code_' + property_df['countrySubd'].loc[0]
    user_df[column] = 1
    return user_df

In [102]:
user_df = add_state(prop_2, user_df)

In [103]:
def user_inputs(user_df):
    price = float(input('Property price?'))
    preapproval = input('Are you preapproved? (0=NO, 1=YES)')
    downpayment_perc = float(input('Downpayment percent: (.25)'))
    interest_rate = float(input('Interest Rate (4.5)')) 
    loan_term = int(input('Loan term in months?'))
    interest_only_payment = input('Interest only? (1=NO, 0=YES)')
    loan = int(input('Enter loan type (1 = Conv, 2 = FHA, 3 = VA, 4 = FSA/RHS)'))
    balloon_payment = input('Balloon payment(1=NO, 0=YES)')
    occupancy_type = input('Will this property be your primary residence? (1=YES, 0=NO)')
    income = int(input('Enter your annual income'))
    submission_of_application = input('Are you working directly with a bank? (1=NO, 0=YES)')
                 
    loan_amount = price * (1-downpayment_perc)
    combined_loan_to_value_ratio = loan_amount / price
    income = income/1000
                 
    user_df['preapproval'] = int(preapproval)
    user_df['loan_amount'] = loan_amount
    user_df['combined_loan_to_value_ratio'] = combined_loan_to_value_ratio   
    user_df['interest_rate'] = interest_rate
    user_df['loan_term'] = int(loan_term) 
    user_df['interest_only_payment'] = int(interest_only_payment) 
    user_df['balloon_payment'] = int(balloon_payment)
    user_df['property_value'] = int(price)
    user_df['occupancy_type'] = int(occupancy_type)
    user_df['income'] = int(income)             
    user_df['submission_of_application'] = int(submission_of_application)   
    
    if loan == 2:
        user_df['loan_type_2'] = 1
    elif loan == 3:
        user_df['loan_type_3'] = 1
    elif loan == 4:
        user_df['loan_type_4'] = 1
    return user_df

# def add_user(user):
#     return user_df.append(user, ignore_index=True)

In [104]:
user_df = user_inputs(user_df)

Property price?40000000
Are you preapproved? (0=NO, 1=YES)0
Downpayment percent: (.25).01
Interest Rate (4.5)9
Loan term in months?200
Interest only? (1=NO, 0=YES)1
Enter loan type (1 = Conv, 2 = FHA, 3 = VA, 4 = FSA/RHS)1
Balloon payment(1=NO, 0=YES)1
Will this property be your primary residence? (1=YES, 0=NO)1
Enter your annual income10000
Are you working directly with a bank? (1=NO, 0=YES)1


In [105]:
user_df.columns

Index(['county_code', 'census_tract', 'preapproval', 'loan_amount',
       'combined_loan_to_value_ratio', 'interest_rate', 'loan_term',
       'interest_only_payment', 'balloon_payment', 'property_value',
       'occupancy_type', 'total_units', 'income', 'submission_of_application',
       'tract_population', 'ffiec_msa_md_median_family_income',
       'tract_to_msa_income_percentage', 'tract_owner_occupied_units',
       'tract_one_to_four_family_homes', 'tract_median_age_of_housing_units',
       'state_code_AL', 'state_code_AR', 'state_code_AZ', 'state_code_CA',
       'state_code_CO', 'state_code_CT', 'state_code_DC', 'state_code_DE',
       'state_code_FL', 'state_code_GA', 'state_code_GU', 'state_code_HI',
       'state_code_IA', 'state_code_ID', 'state_code_IL', 'state_code_IN',
       'state_code_KS', 'state_code_KY', 'state_code_LA', 'state_code_MA',
       'state_code_MD', 'state_code_ME', 'state_code_MI', 'state_code_MN',
       'state_code_MO', 'state_code_MS', 'state_code

In [111]:
user_df[['county_code', 'census_tract', 'preapproval', 'loan_amount',
       'combined_loan_to_value_ratio', 'interest_rate', 'loan_term',
       'interest_only_payment', 'balloon_payment', 'property_value',
       'occupancy_type', 'total_units', 'income', 'submission_of_application',
       'tract_population', 'ffiec_msa_md_median_family_income',
       'tract_to_msa_income_percentage', 'tract_owner_occupied_units',
       'tract_one_to_four_family_homes', 'tract_median_age_of_housing_units',
        'loan_type_2', 'loan_type_3', 'loan_type_4']]

,county_code,census_tract,preapproval,loan_amount,combined_loan_to_value_ratio,interest_rate,loan_term,interest_only_payment,balloon_payment,property_value,...,submission_of_application,tract_population,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,loan_type_2,loan_type_3,loan_type_4
0,08059,8059010901,0,39600000.0,0.99,9.0,200,1,1,40000000,...,1,4410,89900.0,103.0,1320.0,1891.0,45.0,0,0,0


In [106]:
#scale user data
user_scaled = ss.transform(user_df.drop(columns='county_code'))
#data.drop(columns=['lei','debt_to_income_ratio','denial_reason_1','approved','county_code']

In [109]:
prediction = model.predict(user_scaled)

In [110]:
prediction

array([[0., 1.]], dtype=float32)

In [108]:
model.predict_proba(user_scaled)

array([[0., 1.]], dtype=float32)